In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
import numpy
import scipy
import matplotlib as mpl
import matplotlib.dates as mpd
import pylab as plt
import datetime
#
import re
#
import json
import netCDF4
#

In [13]:
class NML(object):
    def nml_to_json(self, nml_in=None, json_out=None):
        '''
        # convert nml to dict; save dict as class member; if json_out is not None,
        # export json to json_out
        #
        # @nml_in: filenname of input nml
        # @json_out: filename of output json file.
        #. 
        '''
        #
        if nml_in is None:
            nml_in = self.nml
        #
        nml_dict = {}
        group_name = ''
        #k_group = 0
        #
        values_out = {}
        ky = ''
        
        with open(nml_in) as fin:
            for rw in fin:
                rw = rw.strip()
                #
                # TODO: maye we should retain comments? if so, we'll need to switch the nested
                #. structure to a list (-like), instead of a dict., to allow for multiple commented-out
                #  entries.
                #
                # for a mid-line comment:
                rw = rw.split('!')[0]
                #
                if len(rw)==0 or rw[0] in ['!', '/', '\n']:
                    continue
                #
                if rw[0]=='&':
                    # new group:
                    if not group_name == '':
                        nml_dict[group_name]=values_out
                    #
                    group_name = rw[1:].strip()
                    values_out = {}
                    #if not group_name in nml_dict.keys():
                    #    nml_dict[group_name]={}
                    continue
                    #
                
                #
                #print('** ', rw)
                #
                # TODO: this logic is almost right, and will probably work most of the time, but
                #. it might be better to be more robust about allowing multi-line entries. Note
                #. this will not work properly for a multi-entry line that end in a multi-line entry.
                ky_vl = rw.split('=')
                if len(ky_vl) == 1:
                    #print('*** debug: ', val, ky_vl)
                    val = val + ky_vl[0]
                    continue
                elif len(ky_vl) == 2:
                    if not ky=='':
                        values_out[ky]=val.strip()
                    ky,val = [s.strip() for s in ky_vl]
                elif len(ky_vl) > 2:
                    # there are multiple entries, presumably separated by commas?? so
                    # key=val,key=val,key=val...
                    #print('** DEBUG: ', [s.strip() for s in re.split('=|,', rw) if not s.strip()==''])
                    values_out.update(dict(numpy.reshape([s.strip()
                                        for s in re.split('=|,', rw) if not s.strip()==''], (-1,2))))
                
            #
        #
        self.nml_dict=nml_dict
        #
        if not json_out is None:
            with open(json_out, 'w') as fout:
                json.dump(nml_dict, fout)
            #
        #
        return nml_dict
    def nml_to_json_depricated(self, nml_in=None, json_out=None):
        '''
        # convert nml to dict; save dict as class member; if json_out is not None,
        # export json to json_out
        #
        # @nml_in: filenname of input nml
        # @json_out: filename of output json file.
        #. 
        '''
        #
        if nml_in is None:
            nml_in = self.nml
        #
        nml_dict = {}
        group_name = ''
        #k_group = 0
        #
        value_out = ''
        ky = ''
        with open(nml_in) as fin:
            for rw in fin:
                rw = rw.strip().split()
                #
                if len(rw)==0 or rw[0] in ['/', '\n', '!']:
                    continue
                #
                if rw[0]=='&':
                    # new group:
                    #group_name = rw[1:].replace('\n', '')
                    group_name = rw[1:].strip()
                    if not group_name in nml_dict.keys():
                        nml_dict[group_name]={}
                    continue
                    #
                #
                #print('** ', rw)
                if '=' in rw:
                    if ky != '':
                        nml_dict[group_name][ky] = value_out
                        #
                    ky,value_out =(s.strip() for s in rw.split('='))
                    #
                else:
                    value_out = value_out + rw.strip()
            #
        #
        self.nml_dict=nml_dict
        #
        if not json_out is None:
            with open(json_out, 'w') as fout:
                json.dump(nml_dict, fout)
            #
        #
        return nml_dict
    #
    def json_to_nml(self, json_in=None, nml_out='input.nml', indent=None):
        '''
        # convert json or dict to an nml. export to nml_out.
        '''
        #
        if json_in is None:
            json_in = self.nml_dict
        #
        if indent is None:
            indent=''
            for k in range(4):
                indent = indent + chr(32)
            #
        #
        if isinstance(json_in, str):
            with open(json_in, 'r') as fin:
                json_in = json.load(fin)
            #
        #
        with open(nml_out) as fout:
            for group,entries in json_in.items():
                fout.write('&{}\n'.format(group))
                #
                for entry,val in entries.items():
                    fout.write('{}{}={}'.format(indent, entry, val ))
                #
            #
            
                    

class NML_from_nml(NML):
    def __init__(self, input_nml):
        #
        super(NML_from_nml,self).__init__()
        #
        with open(input_nml) as fin:
            self.nml = fin.read()
        #
        self.nml_json = self.nml_to_json(input_nml)
        
    #
class NML_from_json(NML):
    def __init__(self, input_json, nml_out):
        #
        super(NML_from_json,self).__init__()
        #
        with open(input_json) as fin:
            self.nml_json = json.load(fin)
        #
        self.nml = self.json_to_nml(self.nml_json, nml_out)
        
    #


In [14]:
JJ = NML_from_nml('input_yoder_v101.nml')


In [15]:
# for ky,vl in JJ.nml_dict.items():
#     print('** {}: {}'.format(ky,vl))


for ky in ['coupler_nml','coupler_nml', 'vegn_data_nml', 'simple_sulfate_nml' ]:
    print('*** {}:{}\n'.format(ky, JJ.nml_dict[ky]))

*** coupler_nml:{'force_donner_moist_conserv': '.false.,', 'months': '1,', 'days': '0,', 'current_date': '1979,1,1,0,0,0,', 'calendar': "'julian'", 'dt_atmos': '1800,', 'dt_cpld': '7200,', 'use_lag_fluxes': '.true.', 'concurrent': '.false.', 'do_ocean': '.false.', 'ocean_npes': '0', 'atmos_npes': '48', 'atmos_nthreads': '1', 'use_hyper_thread': '.false.'}

*** coupler_nml:{'force_donner_moist_conserv': '.false.,', 'months': '1,', 'days': '0,', 'current_date': '1979,1,1,0,0,0,', 'calendar': "'julian'", 'dt_atmos': '1800,', 'dt_cpld': '7200,', 'use_lag_fluxes': '.true.', 'concurrent': '.false.', 'do_ocean': '.false.', 'ocean_npes': '0', 'atmos_npes': '48', 'atmos_nthreads': '1', 'use_hyper_thread': '.false.'}

*** vegn_data_nml:{'nom_ratio': '0.7', 'vegn_to_use': "'uniform'", 'K1': '10,', 'K2': '0.1,', 'fsc_liv': '0.9,', 'fsc_wood': '0.45,', 'c1(4)': '0.3', 'c2(4)': '0.3', 'Vmax': '2.0E-5, 2.0E-5, 2.0E-5, 2.0E-5, 1.50E-5,', 'm_cond': '4., 9., 9., 7., 7.,', 'alpha_phot': '0.05, 0.06, 0.06

In [17]:
print(JJ.nml_dict['aerosolrad_package_nml']['sulfate_indices'])

30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,30,35,35,35,35,35,40,40,40,40,40,45,45,45,45,45,50,50,50,50,50,55,55,55,55,55,60,60,60,60,60,65,65,65,65,65,70,70,70,70,70,75,75,75,75,75,80,80,80,80,82,82,84,84,86,86,88,88,90,91,92,93,94,95,96,97,97,97,97,
